In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from libreco.data import random_split, DatasetPure
from libreco.algorithms import NCF  # pure data, 
from libreco.evaluation import evaluate

Instructions for updating:
non-resource variables are not supported in the long term


In [17]:
anime = pd.read_csv("anime.csv")
rating = pd.read_csv("rating.csv")


In [10]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


we will try to build our model without considering the no ratings

In [18]:
rating = rating[rating.rating != -1]
rating.head()

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10


now we will change the movie data frame to from labels to 1 hot vectors for the genres and type

In [19]:
anime = anime.dropna()
anime = anime.reset_index(drop=True)
anime["genre"] = anime["genre"].apply(lambda x: x.split(", "))
anime["type"] = anime["type"].apply(lambda x: [x])
anime.head()

   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",[Movie],1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",[TV],64,9.26,793665
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samurai, ...",[TV],51,9.25,114262
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",[TV],24,9.17,673572
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",[TV],51,9.16,151266


In [20]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",[Movie],1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",[TV],64,9.26,793665
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samurai, ...",[TV],51,9.25,114262
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",[TV],24,9.17,673572
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",[TV],51,9.16,151266


In [23]:
#changing the genre to a one hot vector
genre = anime["genre"].explode().unique()
genre = pd.get_dummies(genre)
anime = pd.concat([anime, genre], axis=1)
anime = anime.drop(columns=["genre"])
anime.head()


,anime_id,name,type,episodes,rating,members,Action,Adventure,Cars,Comedy,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,32281,Kimi no Na wa.,[Movie],1,9.37,200630,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,5114,Fullmetal Alchemist: Brotherhood,[TV],64,9.26,793665,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,28977,Gintama°,[TV],51,9.25,114262,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,9253,Steins;Gate,[TV],24,9.17,673572,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,9969,Gintama&#039;,[TV],51,9.16,151266,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [24]:
#same for type 
type = anime["type"].explode().unique()
type = pd.get_dummies(type)
anime = pd.concat([anime, type], axis=1)
anime = anime.drop(columns=["type"])
anime.head()

,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,...,Thriller,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV
0,32281,Kimi no Na wa.,1,9.37,200630,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,5114,Fullmetal Alchemist: Brotherhood,64,9.26,793665,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,28977,Gintama°,51,9.25,114262,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,9253,Steins;Gate,24,9.17,673572,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,9969,Gintama&#039;,51,9.16,151266,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [32]:
#preparing the data for the model changing bollean to numbers 
anime["episodes"] = anime["episodes"].apply(lambda x: float(x) if x != "Unknown" else 0)
anime["rating"] = anime["rating"].apply(lambda x: float(x) if x != "Unknown" else 0)
anime["members"] = anime["members"].apply(lambda x: float(x))
#also for genre



TypeError: cannot convert the series to <class 'float'>

In [34]:
#defining a function that will do all the preprocessing
def preprocess_data(anime, rating):
    anime = anime.dropna()
    anime = anime.reset_index(drop=True)
    anime["genre"] = anime["genre"].apply(lambda x: x.split(", "))
    anime["type"] = anime["type"].apply(lambda x: [x])
    genre = anime["genre"].explode().unique()
    genre = pd.get_dummies(genre)
    anime = pd.concat([anime, genre], axis=1)
    anime = anime.drop(columns=["genre"])
    type = anime["type"].explode().unique()
    type = pd.get_dummies(type)
    anime = pd.concat([anime, type], axis=1)
    anime = anime.drop(columns=["type"])
    anime["episodes"] = anime["episodes"].apply(lambda x: float(x) if x != "Unknown" else 0)
    anime["rating"] = anime["rating"].apply(lambda x: float(x) if x != "Unknown" else 0)
    anime["members"] = anime["members"].apply(lambda x: float(x))
    rating = rating[rating.rating != -1]
    return anime, rating

In [35]:
anime1= pd.read_csv("anime.csv")
rating1 = pd.read_csv("rating.csv")
anime1, rating1 = preprocess_data(anime1, rating1)

In [37]:
rating1.head()

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10


In [41]:
#splittin the data into train and test
data = rating1.sample(frac=1)

In [44]:
train_data, test_data = data[:int(0.8*len(data))], data[int(0.8*len(data)):]

In [66]:
unique_movie_titles = np.unique(list(anime1["anime_id"]))
unique_user_ids = np.unique(list(rating1["user_id"]))
unique_movie_titles = [str(i) for i in unique_movie_titles]
unique_user_ids = [str(i) for i in unique_user_ids]



In [65]:
unique_movie_titles

['1',
 '5',
 '6',
 '7',
 '8',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '141',
 '142',
 '143',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '15

In [84]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfd
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs
    user_id =np.array(user_id)
    movie_title = np.array(movie_title)
    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    
    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [85]:
model = RankingModel()

model.call((["1"], ["1"]))

<tf.Tensor 'sequential_38_1/dense_38_1/Add:0' shape=(1, 1) dtype=float32>

In [87]:
import tensorflow_recommenders as tfrs

task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [96]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["anime_id"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [97]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [91]:
cached_train = train_data.batch(8192).cache()
cached_test = test_data.batch(4096).cache()

AttributeError: 'DataFrame' object has no attribute 'batch'

In [98]:
model.fit(train_data, epochs=3)

Epoch 1/3


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.

In [95]:
train_data

,user_id,anime_id,rating
6087150,57110,431,8
5338830,50586,221,6
4258572,40338,19221,7
3222833,29778,13601,10
2421377,23108,9919,9
...,...,...,...
2591992,24520,18661,9
1451692,13914,4181,10
3966711,37539,30187,9
5716384,53591,65,8


In [99]:
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Load data on movie ratings.
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

2024-05-11 19:00:47.039017: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /Users/yassine/tensorflow_datasets/movielens/100k-movies/0.1.1.incomplete8ODZIT/movielens-train.tfre…

Dataset movielens downloaded and prepared to /Users/yassine/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [105]:
print(ratings)

<_PrefetchDataset element_spec={'bucketized_user_age': TensorSpec(shape=(), dtype=tf.float32, name=None), 'movie_genres': TensorSpec(shape=(?,), dtype=tf.int64, name=None), 'movie_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'raw_user_age': TensorSpec(shape=(), dtype=tf.float32, name=None), 'timestamp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'user_gender': TensorSpec(shape=(), dtype=tf.bool, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_occupation_label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'user_occupation_text': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None), 'user_zip_code': TensorSpec(shape=(), dtype=tf.string, name=None)}>


In [108]:
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#creating a sparse matrix
def create_matrix(data, user_col, item_col, rating_col):
    le_user = LabelEncoder()
    le_item = LabelEncoder()
    data["user"] = le_user.fit_transform(data[user_col])
    data["item"] = le_item.fit_transform(data[item_col])
    num_users = data["user"].nunique()
    num_items = data["item"].nunique()
    matrix = sp.coo_matrix((data[rating_col], (data["user"], data["item"])), shape=(num_users, num_items))
    return matrix, le_user, le_item


matrix, le_user, le_item = create_matrix(rating1, "user_id", "anime_id", "rating")


LabelEncoder()


In [1]:
#calculating the sparsity of the matrix
def calculate_sparsity(matrix):
    return 1 - matrix.nnz / (matrix.shape[0] * matrix.shape[1]) 

sparcity = calculate_sparsity(matrix)
print(matrix.nnz)
print(sparcity)
print(matrix.shape[0] * matrix.shape[1]) 


NameError: name 'matrix' is not defined

In [4]:
import pandas as pd
rating1= pd.read_csv("rating.csv")
rating1 = rating1.rename(columns={"user_id": "user", "anime_id": "item", "rating": "label"})


In [5]:
rating1 = rating1[rating1.label != -1]
rating1 = rating1.dropna()
rating1

,user,item,label
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [6]:
import numpy as np
import pandas as pd
from libreco.data import random_split, DatasetPure
from libreco.algorithms import LightGCN  # pure data, algorithm LightGCN
from libreco.evaluation import evaluate

#data = pd.read_csv("examples/sample_data/sample_movielens_rating.dat", sep="::",names=["user", "item", "label", "time"])

data  = rating1
#changing the 

# split whole data into three folds for training, evaluating and testing
train_data, eval_data, test_data = random_split(data, multi_ratios=[0.8, 0.1, 0.1])

train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)
print(data_info)  

Instructions for updating:
non-resource variables are not supported in the long term
n_users: 68861, n_items: 9734, data density: 0.7564 %


In [7]:
train_data, eval_data, test_data = random_split(data, multi_ratios=[0.8, 0.1, 0.1])

In [8]:
train_data, data_info= DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)

In [10]:

from libreco.algorithms import NCF  # pure data, 

ncf = NCF(
    task="rating",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=16,
    n_epochs=10,
    lr=1e-3,
    batch_size=2048,
    num_neg=1,
)

In [11]:
#pip install LibRecommender
import numpy as np
import pandas as pd

from libreco.evaluation import evaluate
# monitor metrics on eval data during training
ncf.fit(
    train_data,
    neg_sampling=False, #for rating, this param is false else True
    verbose=2,
    eval_data=eval_data,
    metrics=["loss"],
)

# do final evaluation on test data
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["loss"],
)
#for implicit feedback, metrics like precision@k, recall@k, ndcg can be used

Training start time: 2024-05-12 15:32:29


AttributeError: `batch_normalization` is not available with Keras 3.